In [1]:
import pandas as pd
import time
import json
import requests
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

#https://www.epochconverter.com/
primeiro = 1692414000
base = 86400
num_dias = 180

In [2]:
cidades = {'sao_luis':'7188',
            'maraba':'7200',
            'parauapebas':'7202'}

In [3]:
dias_ano = []
for i in range(num_dias):
    dias_ano.append((base*i) + primeiro)

In [4]:
calendario = pd.DataFrame(dias_ano, columns=['timestamp'])
calendario['date'] = calendario['timestamp'].apply(lambda x: datetime.fromtimestamp(x))

In [5]:
calendario.head()

,timestamp,date
0,1692414000,2023-08-19 03:00:00
1,1692500400,2023-08-20 03:00:00
2,1692586800,2023-08-21 03:00:00
3,1692673200,2023-08-22 03:00:00
4,1692759600,2023-08-23 03:00:00


In [6]:
calendario['status'] = ''
calendario['descricaoOrigem'] = ''
calendario['descricaoDestino'] = ''
calendario['descricaoClasse'] = ''
calendario['valorTotal'] = ''
calendario['horaPartidaPrevista'] = ''
calendario['horaChegadaPrevista'] = ''

In [7]:
origem_maraba = cidades["maraba"]
destino_sao_luis = cidades["sao_luis"]
destino_parauapebas = cidades["parauapebas"]

quantidade = 1

In [8]:
def consulta_passagens(calendario, origem, destino):
    s = requests.session()
    url = "https://tremdepassageiros.vale.com/sgpweb/rest/externo/VendaInternet/publico/pesquisaPassagem"

    for index, row in calendario.iterrows():
        time.sleep(5)
        payload = {"codigoFerrovia":"04",
                   "codigoLocalOrigem":origem,
                   "codigoLocalDestino":destino,
                   "dataIda": str(int(row['timestamp'])) + '000',
                   "detalheVenda":[{"detalhe":20,
                                    "qtd":1,
                                    "funcionario":False}]}

        r = s.request('POST', url=url,  json=payload)

        try:
            answer = r.json()

            calendario['descricaoOrigem'][index] = str(answer['descricaoOrigem'])
            calendario['descricaoDestino'][index] = str(answer['descricaoDestino'])
            calendario['descricaoClasse'][index] = str(answer['passagensIda'][0]['descricaoClasse'])
            calendario['valorTotal'][index] = str(answer['passagensIda'][0]['valorTotal'])
            calendario['horaPartidaPrevista'][index] = str(answer['passagensIda'][0]['horaPartidaPrevista'])
            calendario['horaChegadaPrevista'][index] = str(answer['passagensIda'][0]['horaChegadaPrevista'])
            calendario['status'][index] = 'Possui Passagem'
        except:
            calendario['status'][index] = 'Não Possui Passagem'
            calendario['descricaoOrigem'][index] = ''
            calendario['descricaoDestino'][index] = ''
            calendario['descricaoClasse'][index] = ''
            calendario['valorTotal'][index] = ''
            calendario['horaPartidaPrevista'][index] = ''
            calendario['horaChegadaPrevista'][index] = ''
            #print(f"Erro ao consultar passagem dia {row['date'].strftime('%d-%m-%Y')}")
    return calendario

In [9]:
maraba_parauapebas = consulta_passagens(calendario, origem_maraba, destino_parauapebas)
maraba_são_luis = consulta_passagens(calendario, origem_maraba, destino_sao_luis)
parauapebas_são_luis = consulta_passagens(calendario, destino_parauapebas, destino_sao_luis)
parauapebas_maraba = consulta_passagens(calendario, destino_parauapebas, origem_maraba)
são_luis_parauapebas = consulta_passagens(calendario, destino_sao_luis, destino_parauapebas)
são_luis_maraba = consulta_passagens(calendario, destino_sao_luis, origem_maraba)

In [12]:
with pd.ExcelWriter('Relatório de Passagem - Trem de passageiros.xlsx') as writer:
    maraba_são_luis.to_excel(writer, sheet_name="Marabá - São Luís", index=False)
    maraba_parauapebas.to_excel(writer, sheet_name="Marabá - Parauapebas", index=False)
    parauapebas_maraba.to_excel(writer, sheet_name="Parauapebas - Marabá", index=False)
    parauapebas_são_luis.to_excel(writer, sheet_name="Parauapebas - São Luís", index=False)
    são_luis_maraba.to_excel(writer, sheet_name="São Luís - Marabá", index=False)
    são_luis_parauapebas.to_excel(writer, sheet_name="São Luís - Parauapebas", index=False)